In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/z/Comp/lu_group/Members/yixuan/Software/HYFA')

In [3]:
from train_gtex import *
# import umap
# import matplotlib.pyplot as plt
# import seaborn as sns
from src.train_utils import forward
from src.distributions import *
from src.eval_utils import *
from src.baselines import *
from tqdm import tqdm
# from matplotlib_venn import venn3, venn3_circles
# import blitzgsea as blitz
# from gseapy.plot import barplot, dotplot
# from gseapy.plot import gseaplot
# import gseapy as gp
# from src.plot_utils import enr_barplot
# import matplotlib
# from matplotlib.ticker import MultipleLocator
import torch
# import os
# import networkx as nx
# from collections import Counter
# sns.set_style("whitegrid")

In [4]:
target_tissue = 'Brain_Cortex'
out = '/z/Comp/lu_group/Members/yixuan/sexde/result/hyfa'

In [5]:
# Parameters
target_tissue = "Adipose_Visceral_Omentum"
out = "./result/hyfa"


In [6]:
RESULTS_DIR = out
MODEL_PATH = '/z/Comp/lu_group/Members/yixuan/Software/HYFA/data/normalised_model_default.pth'
GTEX_FILE = '/z/Comp/lu_group/Members/yixuan/Software/HYFA/data/GTEX_data.csv'
METADATA_FILE = '/z/Comp/lu_group/Members/yixuan/Software/HYFA/data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt'

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', dest='config', default='/z/Comp/lu_group/Members/yixuan/Software/HYFA/configs/default.yaml', type=str)
args, unknown = parser.parse_known_args()

# Initialise wandb
wandb.init(project='multitissue_imputation', config=args.config, mode='disabled')
config = wandb.config
print(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'gpu': 0, 'sweep': False, 'lr': 0.00045568167228053424, 'batch_size': 63, 'epochs': 200, 'patience': 30, 'dropout': 0.17385777664292695, 'bn': True, 'use_demographic_information': True, 'update_edge_attr': False, 'n_graph_layers': 2, 'n_hidden_layers': 1, 'n_hidden_layers_pred': 2, 'n_hidden_layers_prior_var': 0, 'meta_G': 50, 'd_patient': 71, 'd_gene': 48, 'd_edge_attr': 98, 'd_tissue': 120, 'd_edge': 28, 'n_heads': 28, 'beta': 0, 'n_top_genes': None, 'attention_strategy': 'patient', 'metagenes_encoder': 'plain', 'loss_type': 'normal', 'layer': 'gat', 'activation': 'swish', 'norm': 'batch'}


In [8]:
# Load data
adata = GTEx_v8_normalised_adata(file=GTEX_FILE)
adata

Trying to set attribute `.obs` of view, copying.


AnnData object with n_obs × n_vars = 15197 × 12557
    obs: 'Participant ID', 'Tissue', 'Tissue_idx', 'Participant ID_dyn', 'Age', 'Sex'
    var: 'Symbol'
    uns: 'Tissue_dict', 'Sex_dict', 'Tissue_colors'
    obsm: 'Participant ID_feat'
    layers: 'x'

In [9]:
tissue_mapping = {
    'Adipose_Subcutaneous': 'Adipose_Subcutaneous',
    'Adipose_Visceral_Omentum': 'Adipose_Visceral_Omentum',
    'Adrenal_Gland': 'Adrenal_Gland',
    'Artery_Aorta': 'Artery_Aorta',
    'Artery_Coronary': 'Artery_Coronary',
    'Artery_Tibial': 'Artery_Tibial',
    'Brain_Amygdala': 'Brain_Amygdala',
    'Brain_Anterior_cingulate': 'Brain_Anterior_cingulate_cortex_BA24',
    'Brain_Caudate': 'Brain_Caudate_basal_ganglia',
    'Brain_Cerebellar': 'Brain_Cerebellar_Hemisphere',
    'Brain_Cerebellum': 'Brain_Cerebellum',
    'Brain_Cortex': 'Brain_Cortex',
    'Brain_Frontal_Cortex': 'Brain_Frontal_Cortex_BA9',
    'Brain_Hippocampus': 'Brain_Hippocampus',
    'Brain_Hypothalamus': 'Brain_Hypothalamus',
    'Brain_Nucleus': 'Brain_Nucleus_accumbens_basal_ganglia',
    'Brain_Putamen': 'Brain_Putamen_basal_ganglia',
    'Brain_Spinal_cord': 'Brain_Spinal_cord_cervical_c-1',
    'Brain_Substantia_nigra': 'Brain_Substantia_nigra',
    'Breast_Mammary_Tissue': 'Breast_Mammary_Tissue',
    'Cells_Cultured': 'Cells_Cultured_fibroblasts',
    'Cells_EBV': 'Cells_EBV-transformed_lymphocytes',
    'Colon_Sigmoid': 'Colon_Sigmoid',
    'Colon_Transverse': 'Colon_Transverse',
    'Esophagus_Gastro': 'Esophagus_Gastroesophageal_Junction',
    'Esophagus_Mucosa': 'Esophagus_Mucosa',
    'Esophagus_Muscularis': 'Esophagus_Muscularis',
    'Heart_Atrial': 'Heart_Atrial_Appendage',
    'Heart_L_Vent': 'Heart_Left_Ventricle',
    'Kidney_Cortex': 'Kidney_Cortex',
    'Liver': 'Liver',
    'Lung': 'Lung',
    'Minor_Salivary': 'Minor_Salivary_Gland',
    'Muscle_Skeletal': 'Muscle_Skeletal',
    'Nerve_Tibial': 'Nerve_Tibial',
    'Ovary': 'Ovary',
    'Pancreas': 'Pancreas',
    'Pituitary': 'Pituitary',
    'Prostate': 'Prostate',
    'Skin_Not_Sun_Epsd': 'Skin_Not_Sun_Exposed_Suprapubic',
    'Skin_Sun_Epsd': 'Skin_Sun_Exposed_Lower_leg',
    'Small_Intestine': 'Small_Intestine_Terminal_Ileum',
    'Spleen': 'Spleen',
    'Stomach': 'Stomach',
    'Testis': 'Testis',
    'Thyroid': 'Thyroid',
    'Uterus': 'Uterus',
    'Vagina': 'Vagina',
    'Whole_Blood': 'Whole_Blood'
}

adata.obs['Tissue'] = adata.obs['Tissue'].map(tissue_mapping)
adata.uns['Tissue_dict'] = {tissue_mapping[old_key]: value 
                   for old_key, value in adata.uns['Tissue_dict'].items()
                   if old_key in tissue_mapping}


In [10]:
collate_fn = Data.from_datalist

# Split train/val/test
donors = adata.obs['Participant ID'].values
train_donors, test_donors = split_patient_train_test(donors, train_rate=0.8)
train_donors, val_donors = split_patient_train_test(train_donors, train_rate=0.75)
train_mask = np.isin(donors, train_donors)
test_mask = np.isin(donors, test_donors)
val_mask = np.isin(donors, val_donors)

train_dataset = HypergraphDataset(adata[train_mask], disjoint=True, static=False)
val_dataset = HypergraphDataset(adata[val_mask], disjoint=True, static=True)
test_dataset = HypergraphDataset(adata[test_mask], static=True)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, collate_fn=collate_fn, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, collate_fn=collate_fn, shuffle=False)

In [11]:
# Use certain GPU
device = torch.device("cuda:{}".format(config.gpu) if torch.cuda.is_available() else "cpu")

# Select dynamic/static node types
config.update({'static_node_types': {'Tissue': (len(adata.obs['Tissue_idx'].unique()), config.d_tissue),
                            'metagenes': (config.meta_G, config.d_gene)}}, allow_val_change=True)
config.update({'dynamic_node_types': {'Participant ID': (len(adata.obs['Participant ID'].unique()), config.d_patient)}}, allow_val_change=True)

# Model
config.G = adata.shape[-1]
model = HypergraphNeuralNet(config).to(device)  # .double()

In [12]:
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
model.eval()

HypergraphNeuralNet(
  (params): ParameterDict(
      (Tissue): Parameter containing: [torch.FloatTensor of size 49x120]
      (metagenes): Parameter containing: [torch.FloatTensor of size 50x48]
  )
  (metagenes_encoder): PlainEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=12557, out_features=4900, bias=True)
    )
  )
  (hypergraph_layers): ModuleList(
    (0): GATHypergraphLayer(
      (edge_mlp): Sequential(
        (0): Linear(in_features=337, out_features=784, bias=True)
        (1): SiLU()
        (2): Dropout(p=0.17385777664292695, inplace=False)
        (3): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Linear(in_features=784, out_features=784, bias=False)
      )
      (att_agg_mlp): Sequential(
        (0): Linear(in_features=168, out_features=168, bias=False)
        (1): LeakyReLU(negative_slope=0.1)
        (2): Linear(in_features=168, out_features=28, bias=False)
      )
      (dynamic_node_mlp): ModuleDi

#### Store predictions

In [13]:
# added
tissue_dict = adata.uns['Tissue_dict']
tissue_dict_inv = pd.Series({i: v for v, i in tissue_dict.items()})

In [14]:
def create_dataframe(participant_ids, tissue_ids, expression, donor_map, tissue_dict_inv, symbols):
    participant_ids = np.concatenate(participant_ids, axis=0)
    tissue_ids = np.concatenate(tissue_ids, axis=0)
    expression = np.concatenate(expression, axis=0)
    df_metadata = pd.DataFrame({'Participant ID': [donor_map[p] for p in participant_ids],
                               'Tissue': [tissue_dict_inv[t] for t in tissue_ids]})
    df = pd.DataFrame(expression, columns=symbols)
    df = pd.concat([df_metadata, df], axis=1)
    df = df.set_index('Participant ID')
    return df

In [15]:
# all accessible -> target_tissue
donors = adata.obs['Participant ID'].values
lab_donors = adata.obs['Participant ID'].values[adata.obs['Tissue']==target_tissue]
lab_mask = np.isin(donors, lab_donors)

In [16]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)
adata_lab = adata[lab_mask].copy()
cond = adata_lab.obs['Tissue'] == target_tissue

lab_yhat_folds = []

for _, test in kf.split(adata_lab.obs[cond]):
    cond_cp = np.ones(len(adata_lab), dtype=bool)
    cond_cp[np.where(cond)[0][test]] = False
    adata_lab_cp = adata_lab[cond_cp].copy()
    
    lab_cv_dataset = HypergraphDataset(adata_lab_cp, static=True)

    model.eval()
    source_participant_ids = []
    source_tissue_ids = []
    source_expression = []
    target_participant_ids = []
    target_tissue_ids = []
    target_expression = []
    
    for i, d in tqdm(enumerate(lab_cv_dataset)):
        # Set target tissues to missing tissues
        d.target['Tissue'] = torch.tensor([t for t in np.arange(len(tissue_dict)) if t not in d.source['Tissue']])
        d.target['Participant ID'] = torch.zeros_like(d.target['Tissue']) + d.source['Participant ID'][0]
        d.x_target = torch.tensor([-1])  # Unused
        
        # Make predictions
        with torch.no_grad():
            out, node_features = forward(d, model, device, preprocess_fn=None) 
            y_pred = out['px_rate']
        
        # Store
        source_participant_ids.append(d.source['Participant ID'].cpu().numpy() + i)
        source_tissue_ids.append(d.source['Tissue'].cpu().numpy())
        source_expression.append(d.x_source.cpu().numpy())
        target_participant_ids.append(d.target['Participant ID'].cpu().numpy() + i)
        target_tissue_ids.append(d.target['Tissue'].cpu().numpy())
        target_expression.append(y_pred.cpu().numpy())

    # Store data in dataframes
    df_imputed = create_dataframe(target_participant_ids, target_tissue_ids, target_expression,
                                donor_map=lab_cv_dataset.donor_map,
                                tissue_dict_inv=tissue_dict_inv,
                                symbols=adata.var['Symbol'])
    
    lab_yhat_folds.append(df_imputed[df_imputed['Tissue']==target_tissue])
    
lab_yhat = pd.concat(lab_yhat_folds)
    

0it [00:00, ?it/s]

1it [00:00,  5.42it/s]

2it [00:00,  6.47it/s]

3it [00:00,  7.17it/s]

5it [00:00,  9.03it/s]

...

462it [00:38, 12.29it/s]

464it [00:38, 12.62it/s]

466it [00:38, 13.04it/s]

468it [00:38, 13.24it/s]

469it [00:38, 12.10it/s]

In [17]:
dataset = HypergraphDataset(adata, static=True)
model.eval()
source_participant_ids = []
source_tissue_ids = []
source_expression = []
target_participant_ids = []
target_tissue_ids = []
target_expression = []
for i, d in tqdm(enumerate(dataset)):
    # Set target tissues to missing tissues
    d.target['Tissue'] = torch.tensor([t for t in np.arange(len(tissue_dict)) if t not in d.source['Tissue']])
    d.target['Participant ID'] = torch.zeros_like(d.target['Tissue']) + d.source['Participant ID'][0]
    d.x_target = torch.tensor([-1])  # Unused
    
    # Make predictions
    with torch.no_grad():
        out, node_features = forward(d, model, device, preprocess_fn=None) 
        y_pred = out['px_rate']
    
    # Store
    source_participant_ids.append(d.source['Participant ID'].cpu().numpy() + i)
    source_tissue_ids.append(d.source['Tissue'].cpu().numpy())
    source_expression.append(d.x_source.cpu().numpy())
    target_participant_ids.append(d.target['Participant ID'].cpu().numpy() + i)
    target_tissue_ids.append(d.target['Tissue'].cpu().numpy())
    target_expression.append(y_pred.cpu().numpy())

# Store data in dataframes
df_imputed = create_dataframe(target_participant_ids, target_tissue_ids, target_expression,
                              donor_map=dataset.donor_map,
                              tissue_dict_inv=tissue_dict_inv,
                              symbols=adata.var['Symbol'])
df_observed = create_dataframe(source_participant_ids, source_tissue_ids, source_expression,
                               donor_map=dataset.donor_map,
                               tissue_dict_inv=tissue_dict_inv,
                               symbols=adata.var['Symbol'])

lab_y = df_observed[df_observed['Tissue']==target_tissue]
unlab_yhat = df_imputed[df_imputed['Tissue']==target_tissue]

0it [00:00, ?it/s]

1it [00:00,  4.80it/s]

2it [00:00,  6.57it/s]

4it [00:00,  8.99it/s]

6it [00:00,  9.38it/s]

...

828it [01:07, 13.81it/s]

830it [01:07, 13.89it/s]

832it [01:07, 13.88it/s]

834it [01:07, 14.09it/s]

834it [01:07, 12.35it/s]

In [18]:
lab_y.to_csv(f'{RESULTS_DIR}/{target_tissue}.lab_y.csv')
lab_yhat.to_csv(f'{RESULTS_DIR}/{target_tissue}.lab_yhat.csv')
unlab_yhat.to_csv(f'{RESULTS_DIR}/{target_tissue}.unlab_yhat.csv')